In [1]:
from meb import utils, datasets, core, models

from tqdm import tqdm
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import pytorchvideo
import timm
from timm.scheduler.cosine_lr import CosineLRScheduler
import pytorchvideo.transforms
import torch.nn.functional as F



pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

In [18]:
c = datasets.CrossDataset(resize=112, ignore_validation=True, optical_flow=True)
df = c.data_frame
data = c.data

In [19]:
c = datasets.Samm(resize=112, ignore_validation=True, optical_flow=True)
dfs = c.data_frame
datas = c.data

In [20]:
# Add missing AU columns to SAMM that are missing compared to all data
dfs_aus = pd.DataFrame(dfs.loc[:, "AU1":].to_dict(), columns=df.loc[:, "AU1":].columns).fillna(0)
dfs_dropped = dfs.drop(dfs.loc[0, "AU1":].index.tolist(), axis=1)
dfs = pd.concat([dfs_dropped, dfs_aus], axis=1)

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

action_units = dfs.loc[:, "AU1":].columns
df_dataset = dfs
model = GradientBoostingClassifier(random_state=0)

aus = np.array(df_dataset.loc[:, action_units])
emotions = df_dataset["Objective Classes"].astype("str")
model.fit(aus, emotions)
predicted_emotions = model.predict(aus)
f1 = f1_score(emotions, predicted_emotions, average="macro")
print(f1 * 100)

99.17754854925882


In [22]:
df.insert(7, "objective_class", model.predict(np.array(df.loc[:, "AU1":])))

1

#### MEGC2018 validation

In [23]:
idx = df["objective_class"].isin(["6", "7"])
df = df[~idx].reset_index(drop=True)
data = data[~idx]

In [24]:
df["subject"] = df["subject"].astype(str) + df["dataset"]

In [25]:
from meb.core.train_eval import InputData
from typing import List

class MEGC2018ValidatorHDE(core.Validator):
    def __init__(self, config: core.Config, verbose: bool = True):
        super().__init__(config, split_column="dataset")
        self.verbose = verbose
        self.disable_tqdm = False

    def validate(
        self, df: pd.DataFrame, input_data: InputData, seed_n: int = 1
    ) -> List[torch.tensor]:
        utils.set_random_seeds(seed_n)
        dataset_names = ["samm", "casme2"]
        le = LabelEncoder()
        labels = le.fit_transform(df["objective_class"])
        outputs_list = []
        for dataset_name in dataset_names:
            train_metrics, test_metrics, outputs_test = self.validate_split(
                df, input_data, labels, dataset_name
            )
            outputs_list.append(outputs_test)
            if self.verbose:
                self.printer.print_train_test_evaluation(
                    train_metrics, test_metrics, dataset_name, outputs_test.shape[0]
                )
        return outputs_list

In [26]:
from sklearn.preprocessing import OneHotEncoder

def unweighted_average_recall(labels: np.ndarray, preds: np.ndarray) -> float:
    """
    Calculates the unweighted average recall for a multi-class classification task.

    Arguments:
    preds -- binary predictions, numpy array with shape (n_examples, n_classes)
    labels -- binary ground truth labels, numpy array with shape (n_examples, n_classes)

    Returns:
    uar -- unweighted average recall, float
    """
    coder = OneHotEncoder(sparse=False)
    labels = coder.fit_transform(labels.reshape(-1, 1))
    preds = coder.transform(preds.reshape(-1, 1))
    n_examples = labels.shape[0]
    n_classes = labels.shape[1]
    recall_per_class = []
    for class_idx in range(n_classes):
        true_positive = np.sum(np.logical_and(preds[:, class_idx] == 1, labels[:, class_idx] == 1))
        false_negative = np.sum(np.logical_and(preds[:, class_idx] == 0, labels[:, class_idx] == 1))
        recall_per_class.append(true_positive / (true_positive + false_negative))
    uar = np.mean(recall_per_class)
    return uar

class UAR(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, labels: torch.Tensor, outputs: torch.Tensor) -> float:
        _, predictions = outputs.max(1)
        return unweighted_average_recall(labels, predictions)

In [11]:
class Config(core.Config):
    action_units = None
    epochs = 200
    batch_size = 128
    evaluation_fn = UAR
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(models.SSSNet, num_classes=5)

In [12]:
out = MEGC2018ValidatorHDE(Config).validate(df, data)

100%|█████████████████████████████████████████| 200/200 [01:20<00:00,  2.49it/s]


Dataset: samm, n=68 | train_mean: 1.0 | test_mean: 0.8313


100%|█████████████████████████████████████████| 200/200 [01:17<00:00,  2.59it/s]


Dataset: casme2, n=194 | train_mean: 1.0 | test_mean: 0.6399


In [13]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018ValidatorHDE(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 200/200 [00:53<00:00,  3.71it/s]


Dataset: samm, n=68 | train_mean: 1.0 | test_mean: 0.6988


100%|█████████████████████████████████████████| 200/200 [00:50<00:00,  3.95it/s]


Dataset: casme2, n=194 | train_mean: 1.0 | test_mean: 0.562


In [27]:
class Config(core.Config):
    action_units = None
    epochs = 200
    batch_size = 32
    evaluation_fn = UAR
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(timm.models.resnet18, num_classes=5, pretrained=True)

In [28]:
out = MEGC2018ValidatorHDE(Config).validate(df, data)

100%|█████████████████████████████████████████| 200/200 [06:34<00:00,  1.97s/it]


Dataset: samm, n=68 | train_mean: 1.0 | test_mean: 0.6114


100%|█████████████████████████████████████████| 200/200 [06:04<00:00,  1.82s/it]


Dataset: casme2, n=194 | train_mean: 1.0 | test_mean: 0.6362


In [29]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018ValidatorHDE(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 200/200 [02:30<00:00,  1.33it/s]


Dataset: samm, n=68 | train_mean: 1.0 | test_mean: 0.4325


100%|█████████████████████████████████████████| 200/200 [02:01<00:00,  1.65it/s]


Dataset: casme2, n=194 | train_mean: 1.0 | test_mean: 0.377


In [30]:
class Config(core.Config):
    action_units = None
    epochs = 200
    batch_size = 32
    evaluation_fn = UAR
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(timm.models.resnet34, num_classes=5, pretrained=True)

In [31]:
out = MEGC2018ValidatorHDE(Config).validate(df, data)

100%|█████████████████████████████████████████| 200/200 [12:44<00:00,  3.82s/it]


Dataset: samm, n=68 | train_mean: 1.0 | test_mean: 0.6789


100%|█████████████████████████████████████████| 200/200 [11:40<00:00,  3.50s/it]


Dataset: casme2, n=194 | train_mean: 0.9973 | test_mean: 0.5663


In [32]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018ValidatorHDE(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 200/200 [03:30<00:00,  1.05s/it]


Dataset: samm, n=68 | train_mean: 0.9545 | test_mean: 0.4397


100%|█████████████████████████████████████████| 200/200 [02:25<00:00,  1.37it/s]


Dataset: casme2, n=194 | train_mean: 1.0 | test_mean: 0.4793


In [33]:
class Config(core.Config):
    action_units = None
    epochs = 200
    batch_size = 32
    evaluation_fn = UAR
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(timm.models.resnet50, num_classes=5, pretrained=True)

In [34]:
out = MEGC2018ValidatorHDE(Config).validate(df, data)

100%|█████████████████████████████████████████| 200/200 [16:59<00:00,  5.10s/it]


Dataset: samm, n=68 | train_mean: 0.9917 | test_mean: 0.6296


100%|█████████████████████████████████████████| 200/200 [15:33<00:00,  4.67s/it]


Dataset: casme2, n=194 | train_mean: 0.9984 | test_mean: 0.6361


In [35]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018ValidatorHDE(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 200/200 [04:21<00:00,  1.31s/it]


Dataset: samm, n=68 | train_mean: 1.0 | test_mean: 0.3833


100%|█████████████████████████████████████████| 200/200 [02:45<00:00,  1.21it/s]


Dataset: casme2, n=194 | train_mean: 1.0 | test_mean: 0.3651


## R(2+1)D

In [4]:
c = datasets.CrossDataset(resize=112, ignore_validation=True, preload=True, color=True)
df = c.data_frame
data = c.data

Loading data: 100%|███████████████████████████| 860/860 [02:21<00:00,  6.09it/s]


In [5]:
c = datasets.Samm(resize=112, ignore_validation=True, preload=True, color=True)
dfs = c.data_frame
datas = c.data

Loading data: 100%|███████████████████████████| 159/159 [01:18<00:00,  2.02it/s]


In [6]:
# Add missing AU columns to SAMM that are missing compared to all data
dfs_aus = pd.DataFrame(dfs.loc[:, "AU1":].to_dict(), columns=df.loc[:, "AU1":].columns).fillna(0)
dfs_dropped = dfs.drop(dfs.loc[0, "AU1":].index.tolist(), axis=1)
dfs = pd.concat([dfs_dropped, dfs_aus], axis=1)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

action_units = dfs.loc[:, "AU1":].columns
df_dataset = dfs
model = GradientBoostingClassifier(random_state=0)

aus = np.array(df_dataset.loc[:, action_units])
emotions = df_dataset["Objective Classes"].astype("str")
model.fit(aus, emotions)
predicted_emotions = model.predict(aus)
f1 = f1_score(emotions, predicted_emotions, average="macro")
print(f1 * 100)

99.17754854925882


In [8]:
df.insert(7, "objective_class", model.predict(np.array(df.loc[:, "AU1":])))

In [9]:
from torchvision import transforms
import torch.nn.functional as F
#interpolate samples with less than 8 frames
n_frames = 8
for i, video in enumerate(data):
    if video.shape[0] < n_frames:
        new_shape = (n_frames,) + video.shape[1:-1]
        video = torch.tensor(video).permute(3, 0, 1, 2).unsqueeze(0).float()
        new_video = F.interpolate(video, size=new_shape, mode="trilinear")
        data[i] = new_video.squeeze(0).permute(1, 2, 3, 0).byte().numpy()

In [10]:
idx = df["objective_class"].isin(["6", "7"])
df = df[~idx].reset_index(drop=True)
data = data[~idx]

In [11]:
df["subject"] = df["subject"].astype(str) + df["dataset"]

In [12]:
# Create a function that returns the model as it needs to be modified
def r2plus1d(num_classes: int):
    model = torchvision.models.video.r2plus1d_18(weights=torchvision.models.video.R2Plus1D_18_Weights.DEFAULT)
    model.fc = nn.Linear(in_features=512, out_features=num_classes)
    return model

In [15]:
class Config(core.Config):
    action_units = None
    epochs = 100
    batch_size = 32
    evaluation_fn = UAR
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(r2plus1d, num_classes=5)
    loss_scaler = torch.cuda.amp.GradScaler
    channels_last = torch.channels_last_3d
    train_transform = {
        "spatial": None,
        "temporal": datasets.UniformTemporalSubsample(8)
    }
    test_transform = {
        "spatial": None,
        "temporal": datasets.UniformTemporalSubsample(8)
    }

In [16]:
out = MEGC2018ValidatorHDE(Config).validate(df, data)

100%|███████████████████████████████████████| 100/100 [1:15:41<00:00, 45.42s/it]


Dataset: samm, n=68 | train_mean: 1.0 | test_mean: 0.329


100%|███████████████████████████████████████| 100/100 [1:08:13<00:00, 40.94s/it]


Dataset: casme2, n=194 | train_mean: 1.0 | test_mean: 0.4486


In [17]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018ValidatorHDE(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 100/100 [12:26<00:00,  7.46s/it]


Dataset: samm, n=68 | train_mean: 0.9799 | test_mean: 0.2


100%|█████████████████████████████████████████| 100/100 [05:04<00:00,  3.05s/it]


Dataset: casme2, n=194 | train_mean: 1.0 | test_mean: 0.2259
